### Colab instructions

In [ ]:
!git clone https://github.com/GioC1810/aml23-ego.git
!pip install omegaconf coloredlogs wandb

In [ ]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/drive')

## Epic kitchen

#### Feature extraction script

The script extract the features for 5, 10 and 25 num frames per clip

In [ ]:
%%bash

splits=("test")
num_frames=(5)
dense_sampling=("True")

for split_type in ${splits[@]}
do
  for frames in ${num_frames[@]}
  do
    for sampling in ${dense_sampling[@]}
    do
        python3 save_feat.py \
            config=configs/I3D_save_feat.yaml \
            dataset.shift=D1-D1 \
            save.num_frames_per_clip.RGB=$frames \
            save.dense_sampling.RGB=$sampling \
            split=$split_type \
            dataset.RGB.data_path=./ek_data/frames 
    done
  done
done

#### Model training

Indicate:
- shift 
- num frames per clip  
- model 
- model name

In [ ]:
%%bash
cd aml23-ego

python3 train_classifier_ek.py name=model_name \
  config=configs/default.yaml \
  dataset.shift=D1-D1 \
  train.num_frames_per_clip.RGB=25 \
  train.dense_sampling.RGB=False \
  models.RGB.model=MLP_classifier

## Action net

#### Action net frame extraction

In [ ]:
%%bash

ffmpeg -i action-net/video/S04_video.mp4 -vf "fps=30,scale=456:256" -q:v 2 action-net/frames/S04_1/frame_%010d.jpg

#### Feature rgb extraction
Indicate:
- num frames per clip
- split (train or test)
- dense sampling (True or False)

In [ ]:
%%bash

splits=("train" "test")
num_frames=(5 10 25)
dense_sampling=("True")

for split_type in ${splits[@]}
do
  for frames in ${num_frames[@]}
  do
    for sampling in ${dense_sampling[@]}
    do
        python3 save_feat_action_net.py \
            config=configs/feature_rgb_extraction.yaml \
            save.num_frames_per_clip.RGB=$frames \
            save.dense_sampling.RGB=$sampling \
            split=$split_type
    done
  done
done

#### Decompress emg features train

In order to push the features we compressed them, this script decompress them

In [18]:
import lzma

with lzma.open('saved_features_an_multimodal/features_emg_allData_train.pkl.xz', 'rb') as f_in:
    with open('saved_features_an_multimodal/features_emg_allData_train.pkl', 'wb') as f_out:
        f_out.write(f_in.read())

#### Split emg spectogram features in order to push it

This is the script to split the features in 3 different files in order to push to the repo

In [5]:
import pickle

def split_dict(filename, output_prefix):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    
    length = len(data['features'])
    third_length = length // 3

    split = [data['features'][0:third_length], data['features'][third_length:2*third_length], data['features'][2*third_length:length]]

    for i in range(1,4):
        with open(output_prefix + f'_{i}.pkl', 'wb') as f:
            pickle.dump(split[i-1], f)

split_dict('saved_features_an/saved_features_an_emg_10s/features_emg_10s_spectrogram_allData_train.pkl', 'saved_features_an/saved_features_an_emg_10s/features_emg_10s_spectrogram_train_allData_split')

#### Decompress and join emg spectogram features

This is the script to execute in order to obtain the joined features

In [2]:
import pickle

def join_spectogram_feature(filename, output_name):
    joined_features = {'features': []}
    
    for i in range(3):
        with open(filename + f'_{i+1}.pkl', 'rb') as f:
            joined_features['features'].extend(pickle.load(f))
            
    
    with open(output_name, 'wb') as f:
        pickle.dump(joined_features, f)
        
join_spectogram_feature('saved_features_an_multimodal/features_emg_spectrogram_train_split', 
                        'saved_features_an_multimodal/features_emg_spectrogram_train.pkl')

#### Benchmarking EMG

To test the different performance of the EMG classifier according to the different combinations of the EMG preprocessing parameters, and to avoid computing locally the features and push them on github here the code to compute them on colab. 

- Preprocessing
- Reformatting the features as the loader expects them


In [ ]:
%%bash
cd aml23-ego/EMG
python EMG_preprocessing.py

In [ ]:
import pickle
import os
import pandas as pd


def create_emg_features(full_data, split, spectrogram):
    #os.chdir('aml23-ego/EMG/')
    full_data = pd.read_pickle(full_data)
    print(len(full_data))
    
    #full_data = full_data[full_data['file'] == 'S04_1.pkl']
    full_data = full_data.rename(columns={'emg_data': 'features_EMG'})

    emg_features = full_data[
        ['uid', 'features_EMG']]
    emg_features = emg_features.to_dict(orient='index')
    emg_features = {'features': list(emg_features.values())}
    print(emg_features['features'][0]['features_EMG'].shape)
    features_name = f'../saved_features_an_multimodal/features_emg_spectogram_allData_{split}.pkl' if spectrogram \
        else f'../saved_features_an_multimodal/features_emg_10fs_10s_30seg_allData_{split}.pkl'

    with open(features_name, 'wb') as f:
        pickle.dump(emg_features, f)

create_emg_features(full_data="./new_emg_data_10fs_10s_30seg_train.pkl", split='train', spectrogram=False)
create_emg_features(full_data="./new_emg_data_10fs_10s_30seg_test.pkl", split='test', spectrogram=False)

#### Model training

###### RGB model
Indicate:
- Dense sampling: to choose which features the model use for training
- num frames per clip: to choose which features the model use for training
- model: which model to use

In [ ]:
%%bash

num_frames=(5 10 25)
dropout=(0 0.6)
sub_sample_num=(2 3 4)

for frames in ${num_frames[@]}
do
  for sub_sample in ${sub_sample_num[@]}
  do
    for drop in ${dropout[@]}
    do
        python3 train_classifier.py name=TRN_drop${drop}_feature_${frames} \
            config=configs/training_rgb.yaml \
            train.dense_sampling.RGB=True \
            train.num_frames_per_clip.RGB=$frames \
            models.RGB.model=TRN_classifier \
            models.RGB.dropout=$drop \
            models.RGB.subsample_num=$sub_sample \
            models.RGB.num_layers=0
    done
  done
done

In [ ]:
%%bash

num_frames=(5 10 25)
dropout=(0 0.6)

for frames in ${num_frames[@]}
do
  for drop in ${dropout[@]}
  do
        python3 train_classifier.py name=LSTM_drop${drop}_feature_${frames} \
            config=configs/training_rgb.yaml \
            train.dense_sampling.RGB=True \
            train.num_frames_per_clip.RGB=$frames \
            models.RGB.model=Lstm_classifier \
            models.RGB.dropout=$drop \
            models.RGB.subsample_num=0 \
            models.RGB.num_layers=1
  done
done

In [ ]:
%%bash

cd aml23-ego
python3 train_classifier_an_rgb.py name=train_model_rgb_lstm_5s \
  config=configs/training_rgb_an.yaml \
  train.dense_sampling.RGB=True \
  train.num_frames_per_clip.RGB=10 \
  dataset.annotations_path=an_annotations/an_annotations_multimodal_5s \
  dataset.RGB.data_path=saved_features_an/saved_features_an_rgb_5s \
  models.RGB.model=Lstm_classifier \
  models.RGB.dropout=0 \
  models.RGB.subsample_num=None \
  models_dir=saved_models \
  models.RGB.num_layers=1 \



###### EMG model

Indicate:
- spectogram_feat: To choose which kind of features the model is train on
- model: according to the kind of modality 
    - spectogram_feat = True -> LeNet5
    - spectogram_feat = False -> EMG_LSTM

In [ ]:
%%bash

cd aml23-ego

python3 train_classifier_multimodal.py name=train_cnn_emg_80fs\
  config=configs/training_emg.yaml \
  train.spectogram_feat=True \
  models.EMG.lr=0.01 \
  models.EMG.weight_decay=1e-6 \
  dataset.EMG.features_name=features_emg_80fs\
  dataset.annotations_path=an_annotations \
  dataset.EMG.data_path=saved_features_an \
  models.EMG.model=LeNet5

#### Multimodal models

##### Late fusion model

###### Testing

Indicate:
- Dense sampling: to choose which features the model use for training
- num frames per clip: to choose which features the model use for training
- model: which model to use (both RGB and EMG)
- spectogram_feat: To choose which kind of features the model is train on


In [ ]:
%%bash

cd aml23-ego
python3 train_classifier_multimodal.py name=multimodal_late \
  config=configs/late_fusion_test.yaml \
  train.dense_sampling.RGB=True \
  train.num_frames_per_clip.RGB=10 \
  train.spectogram_feat=False \
  models.RGB.model=Lstm_classifier \
  models.EMG.model=EMG_LSTM \
  dataset.annotations_path=an_annotations/an_annotations_multimodal_5s \
  dataset.RGB.data_path=saved_features_an/saved_features_an_rgb_5s \
  dataset.EMG.data_path=saved_features_an/saved_features_an_emg_5s \
  resume_from.RGB=saved_models/ActionSense/RGB_LSTM_5s_10frame/action-classifier_RGB_9.pth \
  resume_from.EMG=saved_models/ActionSense/EMG_LSTM_5s_new/action-classifier_EMG_9.pth \
  dataset.EMG.features_name=features_emg_5s \


##### Midl level fusion model

###### Mid level feature extraction

Indicate: 
- Dense sampling: to choose which features the model use for training
- num frames per clip: to choose which features the model use for training
- split (train or test)
- model: which model to use (both RGB and EMG)
- spectogram_feat: To choose which kind of features the model is train on

In [ ]:
%%bash

cd aml23-ego
python3 save_feat_midlevel_multimodal.py name=midlevel_feat \
  config=configs/mid_level_extraction.yaml \
  split=test \
  save.dense_sampling.RGB=True \
  save.num_frames_per_clip.RGB=10 \
  save.spectrogram_feat=False \
  models.RGB.model=Lstm_classifier \
  models.EMG.model=EMG_LSTM \
  dataset.annotations_path=an_annotations/an_annotations_multimodal_5s \
  dataset.RGB.data_path=saved_features_an/saved_features_an_rgb_5s \
  dataset.EMG.data_path=saved_features_an/saved_features_an_emg_5s \
  resume_from.RGB=saved_models/ActionSense/RGB_LSTM_5s_10frame/action-classifier_RGB_9.pth \
  resume_from.EMG=saved_models/ActionSense/EMG_LSTM_5s_new/action-classifier_EMG_9.pth \
  dataset.EMG.features_name=features_emg_5s \

###### Training

Indicate: 
- Dense sampling: to choose which features the model use for training
- num frames per clip: to choose which features the model use for training
- model: which model to use (both RGB and EMG)
- spectogram_feat: To choose which kind of features the model is train on

In [ ]:
%%bash
cd aml23-ego
python3 train_midlevel_classifier.py name=midlevel_classifier_ZZZ_lstm_lstm_5s_max \
  config=configs/multimodal_midLevel_training.yaml \
  train.dense_sampling.RGB=True \
  train.num_frames_per_clip.RGB=10 \
  train.spectogram_feat=False \
  dataset.annotations_path=an_annotations/an_annotations_multimodal_5s \
  models.hidden_size=256 \
  models.FUSION.lr=0.01 \
  models.FUSION.weight_decay=1e-7\
  train.num_iter=800 \
  train.lr_steps=500 \
  models.FUSION.lr_steps=500 \
  #dataset.FUSION.features_name=midlevel_feat_lstm_lstm_5s